# Enterprise Data Science: Covid 19 Data Analysis Project 

# ![CRISP_DM](../reports/figures/CRISP_DM.png)

# Business Understanding

# Determine Business Objectives
The main goal is  tracking the  Corona VIRUS spread across countries and with personal local information

The general knowledge to me is not that important

I would like deep dive local development of the spreading 
    

## Goals

1-Understanding  the data quality

2-Automated  process as much as possible:
    how many clicks do we need to execute the full pipeline
    

## Contraints:

* Each notebook should be left clean and ready for full execution

# Setting Up The Project
Install Cookiecutter  for having clear structure for the project
            


## Data Understanding (Data_gathering)

Gathering  the data from:

1-RKI ;webscrape (webscraping)

2-John Hopkins (GITHUB)

3-REST API services to retreive data

The data for John Hopkins is updating daily, the data is formatted in time series  

# 1.Import Section

In [1]:
#defining all imports
'''all imports should be at the beginning '''
import subprocess                   
import os

import pandas as pd                 
import numpy as np
from scipy import signal                                            # to import the fliter pakage 
from datetime import datetime                                       # day time lib

from sklearn import linear_model                                    # import Scikit Learn library
reg = linear_model.LinearRegression(fit_intercept=True)             # calling object reg .regression is initialize as object

import dash                                                         # for server client communication
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go                                   # install plotly graph objects

import matplotlib as mpl                                            # library for plot figure 
import matplotlib. pyplot as plt                                 

# 2.1.Data Understanding Phase (Getting and Updating The Data) 

In [2]:
#defining the johns hopkins data and automated the gitHUB pull request

def updating_johns_hopkins_data():                         
             #the function description: getting the johns hopkins data and then updating it  automatically 
    ''' first selecting the johns hopkins data ,
    second we will clone the data 
    then we get pull the data from johns jopkins 
    the result will be stored in the predifined csv structure
    
    '''
    git_pull = subprocess.Popen( "git pull" ,        # the subproces LIb is used to access the terminal(using command clone for first time to get the data )
                         cwd = os.path.dirname( '../data/raw/COVID-19/' ),  # defining the data path for the git pull command
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )              # getting back the pipeline standard out and standard error 
    (out, error) = git_pull.communicate()                        # to automated the GITHUB Request
    
    
    print("Error : " + str(error))
    print("out : " + str(out))                          
    
updating_johns_hopkins_data()        #call the function ,once run this cell the get pull is done in automatice way 

Error : b''
out : b'Already up to date.\n'


# 2.2.Checking The Data Format


The data  is stored  in predefined folder,getting the data and show it's format.
we can look at data set by typing in the data frame 


In [3]:
data_path=r'../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)   
pd_raw.head()    # extra info 

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37599,37856,37894,37953,37999,38054,38070,38113,38129,38140
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7812,7967,8119,8275,8427,8605,8759,8927,9083,9195
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,39847,40258,40667,41068,41460,41858,42228,42619,43016,43403
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1024,1024,1045,1045,1045,1060,1060,1098,1098,1124
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2015,2044,2068,2134,2171,2222,2283,2332,2415,2471


# 3.Data pre-processing

Derive the final data structure,the data  is stored  in predefined folder.

Data preparation phase will focus on johns hopkins data set , time series format.

# 3.1. Relational Johns Hopkins Data

In [4]:
def Relational_data_set_JH():
    ''' reconstruction  the COVID data timeseries in a relational data set

    '''
          # defining a new data structure 
    data_path=r'../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
                                    # to define the data path , using r'.. for windows system 
    pd_raw=pd.read_csv(data_path)   # to import the data  and read it ,in time series format

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})    # rename the country/region/state                      

    pd_data_base['state']=pd_data_base['state'].fillna('no')  # to avoid error with NAN valuesin state,some countries doesnt incloud states therefore we write no in that case 

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1) # drop the lat and long axis from the data
                                                          #axis=1 when dropping columns & axis=0 n dropping rows

# T. transpose the matrix  in raw level , country in coloum level
#stack.to bring multi index from columns to Raw base 0 for singel level 
#then name level 0 with data and the confirmed column also 
    pd_relational_final_data_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
#the type of data was string object , we should convert it to date object with using datetime64 (day time lib)

    pd_relational_final_data_model['date']=pd_relational_final_data_model.date.astype('datetime64[ns]')

    pd_relational_final_data_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False) #store data model to processed
    print(' Number of rows stored: '+str(pd_relational_final_data_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_final_data_model.date)))
# number of rows stored >> we process the john data and we stored the 54530  rows 
Relational_data_set_JH()

 Number of rows stored: 58520
 Latest date is: 2020-08-28 00:00:00


# 3.2.Checking The Data Format


In [5]:
data_path=r'../data/processed/COVID_relational_confirmed.csv'
pd_plot=pd.read_csv(data_path,sep=';')   # to look at the  data set after Relational
pd_plot.head()    # extra info 

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,Anguilla,United Kingdom,0.0
2,2020-01-22,Anhui,China,1.0
3,2020-01-22,Aruba,Netherlands,0.0
4,2020-01-22,Australian Capital Territory,Australia,0.0


# 4. Regression Calculation and Full Process Data Set

In [6]:
def Doubling_time_calc_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        slope
        Doubling rate: double
    '''
    #preparation our data for regression 
    y = np.array(in_array)                               # target vector
    X = np.arange(-1,2).reshape(-1, 1)                   # input vector#columns represtent features, raw represent samples

    assert len(in_array)==3                              
    reg.fit(X,y)                                         # prediction and training the model with fit  
    intercept=reg.intercept_                             # adjust the interception point 
    slope=reg.coef_                                      # coefficient , paramtere for slop 

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5): #plug in filter
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''
# to enusre that we will not destroyed index , the inputs data frame is stored  as result ( df_result=df_input)
# adding new column     (df_result[str(column+'_filtered')]=result)

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0)                     # attention with the neutral element 

    result=signal.savgol_filter(np.array(filter_in),         # calling _saving the filtered data  in new data column
                           5, # window size used for filtering for smoothing the data
                           1) # polynomial degree ,ploynomial 1 is linear  
    df_result[str(column+'_filtered')]=result
    return df_result


def robe_reg(df_input,col='confirmed'):    #write a robe a round function doubling function (rolling function ),
    #rolling move the windows a cross time series 
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame getting dataframe as input
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(                      #slice out the column ,doing the rolling days back ,rolling command move the windows cross time series 
                window=days_back,                      #freezing as a parameter days back 
                min_periods=days_back).apply(Doubling_time_calc_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame
#getting the inputs , calc filter data and merge new features and new calcauating 
results as new column then passing back the large file data
        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])       #cross check that everything is ok (assert command)
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)

    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')

    return df_output.copy()



def calc_doubling_rate(df_input,filter_on='confirmed'):          #current slope ,trend line of doubling the data
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''
#the same as calc_filtered_data function ,first  check that the column all in data set 
    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

# applying groupby function on robe rolling reg filter
#in that case we rename it before merge it back in our input data frame to get our output
    pd_DR_result= df_input.groupby(['state','country']).apply(robe_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output

# full script to process data set

    #REG   object regression( classical Linear Regression)

def final_data_calc():         # small test structure , it mean we get the test data  with array 2 4 6 and push it in our get_doubling_time_via_regression then we can executed 
    test_data_reg=np.array([2,4,6])
    result=Doubling_time_calc_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv(r'../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0]) # reading the large data set 
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()  # sorting the value is important 

    pd_result_larg=calc_filtered_data(pd_JH_data)   # first calc the filter data , we get back new confirmed filter data set
    pd_result_larg=calc_doubling_rate(pd_result_larg) # second calc doubling rate on default parameters 
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered') # final calc doubling rate on confirmed filter 


    mask=pd_result_larg['confirmed']>100  # to clean up doubling rate fiter (avoid NAN values), defining mask vector(0 1 vector) whatever the condition if fullfilled is true otherwise it will be false  
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())
final_data_calc()

the test slope is: [2.]
            date state  country  confirmed  confirmed_filtered  confirmed_DR  \
31895 2020-08-24    no  Germany   236122.0            236214.0    207.718414   
31896 2020-08-25    no  Germany   237583.0            237556.0    152.843207   
31897 2020-08-26    no  Germany   239010.0            239082.4    164.523315   
31898 2020-08-27    no  Germany   240571.0            240582.0    160.009817   
31899 2020-08-28    no  Germany   242126.0            242081.6    154.408858   

       confirmed_filtered_DR  
31895             191.490493  
31896             186.165472  
31897             165.679450  
31898             158.012866  
31899             160.430782  


# 5. Data Visualization

In [ ]:
# creating dashboard and definig its layout and markdown for it. Adding traces and defining  the dictionary 

df_input_large=pd.read_csv(r'../data/processed/COVID_final_set.csv',sep=';')

#generate figure object 
#with that object we can add different traces lines chart styles

fig = go.Figure()      
# starting defining the layout 
app = dash.Dash()
app.layout = html.Div([

    #Defining the header in dashboard
    dcc.Markdown('''
    Applied Data Science on COVID-19 data
    
    The project Goal:
    applying a cross industry standard process to teach data science 
    covering the full walkthrough of : 
                    automated data gathering 
                    data transformations
                    filtering and machine learning to approximating the doubling time
                    static) deployment of responsive dashboard
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

#first dropdown option to select the individual countries 
    #predefine each of possible option in hard coded way,construct our list of dictionaries in one nested loop
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany','Italy','Egypt'], # which are pre-selected
        multi=True    #many of countries can be selceted in our list
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),
    
#second dropdown list for selecting between the data type(confirmed,confirmed_filtered,confirmed_DR,confirmed_filtered_DR)
 # in that dropdown only one is allowed to be choosen  <multi=False>  
    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False  # in that dropdown only one is allowed to be choosen
    ),

    dcc.Graph(figure=fig, id='main_window_slope')   # the figure will be down the dropdown list , writting the command up will creat the figure above
])

#ending defining the layout 

# input and output countrol option 
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):

#Adiing  If function to display the text 'approximated doubling..... on x axis' if we choose the confirmed_filtered_DR',else it will show 'Confirmed infected people' text
    if 'confirmed_filtered_DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)' # deinfe y axis in first selecting
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)' #define y axis in second selecting
              }


    traces = []    #adding traces and defining  the dictionary for it
    for each in country_list:                  #bring in all other countries in the figure therefor we loop it(looping over the individual countries)

        df_plot=df_input_large[df_input_large['country']==each]  

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()

            
#adding traces and defining the dictionary for it ,and  for x axis we go for date and y axis we go show doubling results  
#defines how to plot the trace 
        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',      # change the individual traces (markers,lines,....)
                                opacity=1,                 #the color intense
                                name=each
                        )
                )

    return {                           # retunr a dictionary with all data and traces 
            'data': traces,             # that part for defining the overall layout 
            'layout': dict (            #visualized the layout (size), define the layout properties 
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }


app.run_server(debug=True, use_reloader=False)     #launching the server 


Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
